## ElemNet: A formation energy prediction tool from elemental composition

A formation energy prediction tool using 17-layered deep neural network trained on the Open Quantum Materials Database (OQMD).
<br>
**Input**: Takes a numpy array of shape (number_inp_samples,86) with the rows representing different compounds, and columns representing the elemental compositions from the 86 elements of the set:
<br>
['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu'],
<br> elemental compositon does not contain any element from ['He', 'Ne', 'Ar', 'Po', 'At','Rn','Fr','Ra']
<br>
**Output**: Returns a array of shape (number_inp_samples,1) with the predicted formation energy

In [1]:
import tensorflow as tf
import numpy as np
import time, os, re
from collections import OrderedDict, defaultdict

In [2]:
# here the necessary functions for transforming a list of formulas to input arrays is given

elements = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti', 'V', 
            'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 
            'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 
            'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 
            'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu']

formulare = re.compile(r'([A-Z][a-z]*)(\d*)')

def parse_formula(formula):
    pairs = formulare.findall(formula)
    length = sum((len(p[0]) + len(p[1]) for p in pairs))
    assert length == len(formula)
    formula_dict = defaultdict(int)
    for el, sub in pairs:
        formula_dict[el] += float(sub) if sub else 1
    return formula_dict

def formula2array(formulas):
    samp_input = np.zeros(shape=(len(formulas), 86), dtype=np.float32)
    i = -1
    for formula in formulas:
        i+=1
        keys = formula.keys()
        values = formula.values()
        total = float(sum(values))
        for k in keys:
            samp_input[i][elements.index(k)] = formula[k]/total
    return samp_input

In [3]:
# workflow example
formulas_str = ['H2O','NaCl', 'H2SO4'] # first a list of formulas (strings) must be defined 
formulas = [parse_formula(x) for x in formulas_str] # the "formula strings" are then transformed to dicts 
                                                # where elements are keys and respective element counts the values
print('The input dict for the provided formulas are:')
print(formulas)
inp = formula2array(formulas) # the input dicts are finally translated to input arrays of shape (num_samples,len(elements))
k=0
print(f'sampel input for {formulas_str[k]}:')
print(inp[k])

The input dict for the provided formulas are:
[defaultdict(<class 'int'>, {'H': 2.0, 'O': 1}), defaultdict(<class 'int'>, {'Na': 1, 'Cl': 1}), defaultdict(<class 'int'>, {'H': 2.0, 'S': 1, 'O': 4.0})]
sampel input for H2O:
[0.6666667  0.         0.         0.         0.         0.
 0.33333334 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.       

In [5]:
# now the pretrained model can be loaded for inference
from tensorflow.keras.models import load_model
path = 'sample/model.h5' # path to model
model = load_model(path)

print('The predicted formation energies are:')
preds = np.round(model.predict(inp),3)
{formulas_str[k]:preds[k] for k in range(len(preds))}

The predicted formation energies are:


{'H2O': array([-0.188], dtype=float32),
 'NaCl': array([-2.079], dtype=float32),
 'H2SO4': array([-1.475], dtype=float32)}

In [6]:
# and the workflow as a single function (from formulas to predictions):
def formula2pred(formulas,model):
    formulas = [parse_formula(x) for x in formulas]
    return model.predict(formula2array(formulas))
 
formula2pred(['Cs1Ho1S4Si1'],model)

array([[-1.6301346]], dtype=float32)

In [7]:
# sample compositions (column: comp) and related energies (column: delta_e) can be obatained by the following df 
import pandas as pd

oqmd_data_path = '../training-data/oqmd_all-22Mar18.csv'
oqmd_data = pd.read_csv(oqmd_data_path,sep=r"\s+",engine='python', na_values= 'None',error_bad_lines=False)
oqmd_data.head()

,comp,energy_pa,volume_pa,magmom_pa,bandgap,delta_e,stability
0,Cs1Ho1S4Si1,-5.353489,27.1652,0.000069,3.024,-1.608940,-0.064029
1,Lu1,-4.511592,28.7838,0.046445,0.000,0.012590,0.012590
2,Tm1,-4.468631,29.5370,0.020100,0.000,0.006394,0.006394
3,Ne1,-0.029181,21.7199,NaN,11.910,0.000137,0.000137
4,La1,-4.804203,37.7862,0.582882,0.000,0.131232,0.131232


In [9]:
# the model was trained and tested on the following datasets
from data_utils import load_csv

config = {"loss_type": "mae", "log_file": "sample.log", "config_file": "sample/job.config", "use_valid": False, 
         "test_metric": "mae", "test_data_path": "../training-data/test_set.csv", "label": "delta_e", 
         "project": "ElemNet", "input_types": ["elements_tl"], "train_data_path": "../training-data/train_set.csv", 
         "architecture": "1024x4D-512x3D-256x3D-128x3D-64x2-32x1-1", "save_path":"sample/sample_model","dataset": "OQMD"}

X_train, y_train, X_valid, y_valid, X_test, y_test = load_csv(config['train_data_path'],
                                                              test_data_path=config['test_data_path'],
                                                              input_types=config['input_types'],
                                                              label=config['label'])
# train set
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
# validation set
X_valid = X_valid.astype('float32') 
y_valid = y_valid.astype('float32')
# test set
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

train data path is  ../training-data/train_set.csv
input attribute sets are:  ['elements_tl']
test data path is  ../training-data/test_set.csv
input attributes are:  ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu']
label: delta_e
           energy_pa      volume_pa      magmom_pa        bandgap  \
count  307305.000000  307305.000000  208214.000000  307305.000000   
mean       -5.498033      22.045599       0.419167       0.141143   
std         1.940040       7.951182       0.604534       0.676448   
min       -13.575205       4.149110     

In [10]:
# finally, we can evaluate the model performance (with respect to MAE) on the test set

def test_model(model,X,y, batch_size=100):
    count=0
    abs_error = 0
    while count*batch_size<len(y)-1:
        ind = count*batch_size
        pred = model.predict(X[ind:ind+batch_size])
        abs_error += np.abs(pred.squeeze()-y[ind:ind+batch_size]).sum()
        count+=1
    return abs_error/len(y)
 
mae = test_model(model,X_test,y_test)
mae

0.04863117278486311

In [12]:
# To get a feeling for the model performance, the average absolute deviation between ground truth samples and their mean should be helpful:
dev = np.abs(y_test-y_test.mean()).mean()
print(dev)
# as well as the mae of the model predictions divided by this quantity
mae/dev

0.58809125


0.08269324264680687